# Completion Rate (CR) per client - Tasa de Finalización

## Carga de datos

In [1]:
# --------------------------------------------------
# PASO 1: IMPORTAR LIBRERÍAS
# --------------------------------------------------
import pandas as pd
import numpy as np
import statsmodels.api as sm

# --------------------------------------------------
# PASO 2: CARGAR DATOS
# --------------------------------------------------
# Ajusta la ruta según tu proyecto:
df = pd.read_csv("../../data/processed/navegacion_clientes_experimento_limpio.csv")

## Toma de muestras - por sesión

In [2]:
# --------------------------------------------------
# PASO 2: DEFINIR SI EL CLIENTE (client_id) COMPLETÓ EL PROCESO
# --------------------------------------------------
# Para cada fila, la columna "completed" es 1 si `process_step` == "confirm", 0 en caso contrario.
df["completed"] = df["process_step"].eq("confirm")

# --------------------------------------------------
# PASO 3: AGRUPAR POR (variation, client_id)
# --------------------------------------------------
# Queremos quedarnos con 1 fila por cliente, indicando si completó el proceso (al menos una vez).
df_clients = (
    df.groupby(["variation", "client_id"], as_index=False)
      .agg({"completed": "max"})  
)
# 'completed' = 1 si en alguna sesión del cliente con esa variation hubo un confirm, 0 de lo contrario.
# Forma 2: Lucho/Toño: 
# df_sessions = df_sessions.groupby("client_id"["completed"].max().reset_index()
# Forma 3: numpy intersect by Cris

# (Opcional) Filtrar si tienes una columna "Unknown" que deseas excluir
df_clients = df_clients[df_clients["variation"].isin(["Control","Test"])].copy()

# --------------------------------------------------
# PASO 4: CALCULAR COMPLETION RATE POR GRUPO (Control vs. Test)
# --------------------------------------------------
cr_data = df_clients.groupby("variation")["completed"].agg(["mean", "sum", "count"])

print("=== COMPLETION RATE POR GRUPO (variation) ===")
print(cr_data)
print("\n- mean  = Tasa de finalización (Completion Rate) a nivel de cliente")
print("- sum   = Nº de clientes que completaron al menos una vez")
print("- count = Nº total de clientes en cada grupo\n")

=== COMPLETION RATE POR GRUPO (variation) ===
               mean    sum  count
variation                        
Control    0.641139  10473  16335
Test       0.680411  12838  18868

- mean  = Tasa de finalización (Completion Rate) a nivel de cliente
- sum   = Nº de clientes que completaron al menos una vez
- count = Nº total de clientes en cada grupo



## Hipótesis

In [3]:
# --------------------------------------------------
# PASO 5: TEST DE HIPÓTESIS (Z-TEST DE PROPORCIONES)
# --------------------------------------------------
# H0: CR(Test) = CR(Control)
# H1: CR(Test) != CR(Control) (test bilateral, por ejemplo)

test_sum   = cr_data.loc["Test",    "sum"]
test_count = cr_data.loc["Test",    "count"]
ctrl_sum   = cr_data.loc["Control", "sum"]
ctrl_count = cr_data.loc["Control", "count"]

count = np.array([test_sum, ctrl_sum])
nobs  = np.array([test_count, ctrl_count])

stat, p_value = sm.stats.proportions_ztest(count, nobs, alternative='two-sided')

print("=== PRUEBA DE HIPÓTESIS PARA COMPLETION RATE (a nivel de cliente) ===")
print(f"Estadístico z: {stat:.4f}")
print(f"p-value      : {p_value:.6f}")

=== PRUEBA DE HIPÓTESIS PARA COMPLETION RATE (a nivel de cliente) ===
Estadístico z: 7.7695
p-value      : 0.000000


## Interpretación

In [4]:
alpha = 0.05

if p_value < alpha:
    print(f"\nConclusión: Rechazamos H0 (p < {alpha}).")
    print("Hay diferencia estadísticamente significativa en la Completion Rate entre Test y Control (a nivel de cliente).")
else:
    print(f"\nConclusión: No rechazamos H0 (p >= {alpha}).")
    print("No se detecta diferencia estadísticamente significativa en la Completion Rate entre Test y Control (a nivel de cliente).")

# --------------------------------------------------
# PASO 6: RESUMEN
# --------------------------------------------------
cr_test    = cr_data.loc["Test",    "mean"]
cr_control = cr_data.loc["Control", "mean"]

print(f"\n=== Resumen Final (KPI a nivel de cliente) ===")
print(f"Completion Rate (Test)    : {cr_test:.3%}")
print(f"Completion Rate (Control) : {cr_control:.3%}")
print(f"p-value                   : {p_value:.6f}")


Conclusión: Rechazamos H0 (p < 0.05).
Hay diferencia estadísticamente significativa en la Completion Rate entre Test y Control (a nivel de cliente).

=== Resumen Final (KPI a nivel de cliente) ===
Completion Rate (Test)    : 68.041%
Completion Rate (Control) : 64.114%
p-value                   : 0.000000


## Consideraciones

### Agrupación a nivel de sesión

- Interpretación: “De todas las sesiones, ¿cuántas resultan en confirm (éxito)?”.
- Un mismo cliente con 5 sesiones, 3 de ellas exitosas, contará 3 “éxitos” en el numerador.

### Agrupación a nivel de cliente

- Interpretación: “¿Cuántos clientes, como proporción de los que vieron Test/Control, completaron al menos una vez el proceso?”.
- Esto “colapsa” todos los eventos del cliente: si en alguna sesión completó, completó todo.
- Un mismo cliente con 3 sesiones exitosas igualmente aporta +1 al numerador, no +3.

### Otros niveles o criterios

- A veces se desea solo la primera sesión del cliente (para medir la “primera experiencia”).

## Aclaración del KPI

Lo esencial es definir el KPI con claridad. Si tu objetivo es medir la probabilidad de que un cliente complete el proceso en algún momento, agrupar por client_id tiene sentido. Si en cambio mides la probabilidad de que una sesión acabe en confirm, el agrupamiento por visit_id es correcto.

## Es por cliente

Basándote en la descripción del experimento y en cómo se asignaron los participantes (cada cliente quedó en Control o Test), lo más coherente con la pregunta de negocio – “¿Más clientes llegan a completar el proceso con la nueva interfaz?” – es calcular la tasa de finalización a nivel de cliente.